# Rapport de Projet INF421 : Algorithmes de Planification de Chemin
Auteur : Élève X

Date : Novembre 2025

## 1. Introduction
Ce projet a pour objectif de concevoir, implémenter et évaluer des algorithmes de planification de chemin pour robots ou drones. Le problème fondamental, défini comme "trouver un mouvement sans collision entre une configuration initiale et finale", est ici instancié dans un environnement rectangulaire 2D peuplé d'obstacles rectangulaires. Nous explorons deux heuristiques distinctes : le Particle Swarm Optimization (PSO), une méthode bio-inspirée, et le Rapidly-exploring Random Tree (RRT), une approche basée sur les graphes. Enfin, nous étendons l'étude au cas multi-robots avec contraintes de sécurité.
## 2. Mise en place de l'environnement
### Question 1 : Parsing et Initialisation
Nous avons développé une classe `Environment` et une fonction de parsing. Le fichier d'entrée est lu séquentiellement pour extraire les dimensions $(x_{max}, y_{max})$, les positions de départ/arrivée des deux robots, le rayon de sécurité $R$, et la liste des obstacles $(x_o, y_o, l_x, l_y)$.

Choix d'implémentation : Nous stockons les obstacles sous forme de liste de tuples. Cette structure simple permet une itération rapide pour la détection de collisions, suffisante pour le nombre d'obstacles dans les scénarios de test.
### Question 2 : Affichage
La visualisation est assurée par la bibliothèque `matplotlib`. L'environnement est représenté par une figure où les obstacles sont des `patches.Rectangle` noirs. Les chemins sont superposés sous forme de lignes brisées. Cette méthode permet un débogage visuel immédiat de la validité topologique des chemins.

## 3. Approche PSO (Particle Swarm Optimization)
### Question 3 : Définition des Particules et Fitness
Contrairement à l'optimisation de fonctions classiques où une particule est un point, ici une particule représente un chemin complet.

- **Structure de données** : Une particule est un vecteur de dimension $2 \times N$ correspondant aux coordonnées de $N$ points de passage (waypoints) intermédiaires. Le chemin complet est formé en reliant $Start \rightarrow w_1 \rightarrow ... \rightarrow w_N \rightarrow Goal$.
- **Fonction de Fitness (Objectif)** : Elle est définie comme la somme des distances euclidiennes entre points consécutifs (longueur du chemin) plus une pénalité exponentielle en cas de collision avec un obstacle. La minimisation de cette fonction favorise les chemins courts et valides.

### Question 4 : Détection de Collision
Segment par segment, la méthode `check_collision` vérifie que les extrémités du segment ne sont pas dans l'obstacle, puis fait appel à la fonction `lines_intersect`, pour déterminer de façon exacte si le segment coupe l'un des quatre bords de l'obstacle.

### Question 5 : Pseudo-code 

```Initialiser S particules avec des positions (waypoints) et vitesses aléatoires 
Tant que k < MaxIterations:\\
    Pour chaque particule i:
        Calculer Fitness(x_i)
        Si Fitness(x_i) < Fitness(p_best_i): Mettre à jour p_best_i
        Si Fitness(x_i) < Fitness(g_best): Mettre à jour g_best
    Pour chaque particule i:
        Mettre à jour v_i selon Eq. (1) 
        Mettre à jour x_i selon Eq. (2) 
Retourner g_best 
```

### Question 6 : Complexité
La complexité par itération est $O(S \cdot N \cdot O_{bst})$, où $S$ est le nombre de particules, $N$ le nombre de waypoints, et $O_{bst}$ le nombre d'obstacles (à cause du coût de vérification de collision).

### Question 7 & 8 : Implémentation et Résultats
$\tr$

Déterminer les bons paramètres adaptés à chaque scénario pour obtenir un chemin.

L'implémentation vectorisée avec `numpy` accélère les calculs.

Choix des hyper-paramètres : $c_1 = c_2 = 1.5$ et $w=0.5$ offrent un bon compromis entre exploration (éviter les minima locaux) et exploitation (raffiner le chemin).

Observation : Le PSO de base converge vite mais reste souvent coincé dans des optima locaux (chemins "tendus" mais traversant un obstacle, ou contournant trop largement) car la surface de fitness est très accidentée.

### Question 9, 10, 11 : Améliorations (Restart, Recuit Simulé, Apprentissage Dimensionnel)
- **Random Restart**: Réinitialiser les positions permet de trouver d'autres topologies de chemin (passer à gauche ou à droite d'un obstacle). Cela améliore grandement la robustesse sur les scénarios complexes.
- **Recuit Simulé**: Accepter temporairement des solutions dégradées permet de "sortir" d'un obstacle. $T^0$ a été fixé haut puis décroît ($\beta = 0.95$).
- **Apprentissage Dimensionnel**: Cette technique a permis d'affiner les coordonnées individuelles des waypoints, utile pour "corner" les obstacles de près sans collision.

### Question 12 : Conclusion sur PSO
$\tr$
**Proposer notre propre amélioration**

Le PSO est puissant pour l'optimisation continue mais s'adapte difficilement à la topologie discrète des obstacles (tout ou rien). Il nécessite beaucoup de "tuning" pour éviter les collisions tout en minimisant la distance.

Les problèmes arrivent au scénario 3.

- **pénalité progressive** : en fonction de l'épaisseur du mur traversé. En effet, dans les scénarios deux trois et quatre, les particules ont beaucoup de chance de traverser plusieurs obstacles, mais si elles se déplacent vers la droite ou la gauche, les obstacles étant massids, elle ne détectera pas de changement dans son score. De même, si on la reset aléatoirement, elle rencontrera tout de même de nombreux obstacles.
+ Pénalité en fonction du nombre d'intersection avec son propre chemin pour éviter les dépenses inutiles / chemins non optimaux

- **amélioration du recuit simulé** : on garde tout de même le vrai meilleur score en mémoire, mais ce n'est plus celui qui guide les autres 

- **DLO multi-step** : 

Pour `num_particles=100`, `num_waypoints=15` et `max_iter=150` : 

Sans amélioration :
<div style="display: flex; flex-wrap:wrap; gap:20px;" >
<div style="text-align: center;">
    <img src="assets/SC0-amelioless.png" width="220" height="187">
    <br>
    <em>Scénario 0 sans amélioration</em>
</div>
<div style="text-align: center;">
    <img src="assets/SC1-amelioless.png" width="220" height="187">
    <br>
    <em>Scénario 1 sans amélioration</em>
</div>
<div style="text-align: center;">
    <img src="assets/SC2-amelioless.png" width="220" height="187">
    <br>
    <em>Scénario 2 sans amélioration</em>
</div>
<div style="text-align: center;">
    <img src="assets/SC3-amelioless.png" width="220" height="187">
    <br>
    <em>Scénario 3 sans amélioration</em>
</div>
<div style="text-align: center;">
    <img src="assets/SC4-amelioless.png" width="220" height="187">
    <br>
    <em>Scénario 4 sans amélioration</em>
</div>
</div>

## 4. Approche RRT (Rapidly-exploring Random Tree)
### Question 13 : Structure de données Arbre
L'arbre est constitué de nœuds. Chaque `Node` contient : 
1. Ses coordonnées $(x,y)$.
2. Une référence vers son parent (`parent`).
3. Le coût cumulé depuis la racine (`cost`).

### Question 14 : Reconstruction du chemin
On remonte simplement les pointeurs `parent` depuis le nœud final jusqu'à la racine (start), puis on inverse la liste.

### Question 15 : Pseudo-code RRT (avec Rewiring - RRT*)
```Arbre = {Start}
Pour k = 1 à MaxIterations:
    v_rand = Echantillonner(E) 
    v_nearest = PlusProcheVoisin(Arbre, v_rand)
    v_new = Etendre(v_nearest, v_rand, delta_s) 
    Si PasCollision(v_nearest, v_new):
        Voisins = TrouverVoisins(v_new, delta_r)
        ChoisirMeilleurParent(v_new, Voisins)
        Ajouter v_new à Arbre
        Recabler(Voisins, v_new) [cite: 76]
```

### Question 16 : Complexité
Une itération coûte $O(N)$ pour la recherche du plus proche voisin (sans structure spatiale avancée) et $O(1)$ pour l'extension. La complexité totale est quadratique en nombre de nœuds $O(K^2)$ si non optimisée, ou $O(K \log K)$ avec un KD-Tree.

### Question 17 & 18 : Résultats RRT
Le RRT explore l'espace beaucoup plus efficacement que le PSO pour trouver un chemin valide, même tortueux.

Compromis : Un grand $\delta_s$ accélère l'exploration mais rend difficile la traversée de passages étroits.

### Question 19 : Optimisation de chemin (Smoothing)
Après avoir trouvé un chemin, nous appliquons un lissage : on prend deux nœuds aléatoires du chemin. Si le segment les reliant est libre d'obstacles, on supprime les nœuds intermédiaires. Cela exploite l'inégalité triangulaire. Complexité : Linéaire en nombre d'itérations de lissage.

### Question 21 : Intelligent Sampling
Au lieu d'un échantillonnage uniforme, on biaise la distribution : avec une probabilité $p=0.3$, on tire un point dans une gaussienne centrée sur les frontières des obstacles. Cela force l'arbre à explorer les "goulots d'étranglement".

### Question 22 : Conclusion RRT
Le RRT (surtout sa version RRT*) s'est avéré plus robuste que le PSO pour ce problème géométrique. L'intelligent sampling réduit drastiquement le temps de calcul dans les environnements encombrés ("cluttered").

## 5. Le problème multi-robots
### Question 23 : Algorithme proposé
Pour le problème à 2 robots avec zone de sécurité $R$, nous proposons une **Planification Découplée avec Priorité** :
1. Calculer le chemin optimal pour le Robot 1 (R1) en ignorant R2 (RRT* standard).
2. Calculer le chemin pour le Robot 2 (R2) en considérant R1 comme un obstacle dynamique.
- L'espace-temps est considéré : à l'instant $t$, la position $P_{R1}(t)$ définit un disque interdit de rayon $R$ pour R2.
- Si échec, on utilise une approche centralisée (RRT dans l'espace de configuration 4D : $x_1, y_1, x_2, y_2$).

Pour simplifier l'implémentation demandée : nous avons opté pour le **RRT Centralisé (4D)**.

L'état est $S = (x_1, y_1, x_2, y_2)$.

Condition de validité :
1. $(x_1, y_1)$ hors obstacles.
2. $(x_2, y_2)$ hors obstacles.
3. $\sqrt{(x_1-x_2)^2 + (y_1-y_2)^2} > R$.

### Question 24 : Résultats
L'approche centralisée garantit l'absence de collision mais augmente la dimensionnalité de l'espace de recherche, ralentissant la convergence. Sur les scénarios fournis, les robots réussissent à se croiser en contournant l'un l'autre grâce à la troisième condition.

## 6. Question finale
### Question 25 : Utilisation de l'IA et Github
Pour ce projet, Github a servi au versioning du code, permettant de tester les branches (PSO vs RRT) sans perte de données.

L'IA (modèles LLM comme celui utilisé ici) a été utilisée comme assistant de pair-programming pour :
1. Générer les squelettes de classes (boilerplate code).
2. Vérifier les formules mathématiques (mise à jour de vitesse PSO).
3. Déboguer les erreurs de dimensions `numpy`.

L'IA n'a pas remplacé la conception algorithmique mais a accéléré l'implémentation de la syntaxe et la rédaction du rapport.